In [22]:
%run __init__.py
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import numpy as np
import open3d as o3d

In [24]:
pnp_results = np.load('../calib_output/pnp_results.npy', allow_pickle=True).item()

In [25]:
r_pointclouds = np.load('../calib_output/r_pcd.npy', allow_pickle=True).item()

In [26]:
for key in pnp_results.keys():
    print(pnp_results[key][0]) # PnP transformation T: chessboard -> camera
    print(pnp_results[key][1].shape) # Transformed keypoints from chessboard: T @ kp_chessboard
    print(r_pointclouds[key].shape) # Segmented chessboard from Azure
    break

[[ 0.99127092  0.00252416  0.13181648 -0.01185215]
 [ 0.00531094  0.99824066 -0.0590541  -0.10442502]
 [-0.13173363  0.05923868  0.98951353  1.28062433]
 [ 0.          0.          0.          1.        ]]
(48, 3)
(21730, 3)


In [27]:
import mrob

#initialization
graph = mrob.FGraph()
W = np.array([1])
n1 = graph.add_node_pose_3d(mrob.geometry.SE3())

In [29]:
for key in pnp_results.keys():
    pnp_chessboard_points = pnp_results[key][1]
    
    plane_pointcloud = r_pointclouds[key]
    
    plane_centroid = mrob.registration.estimate_centroid(plane_pointcloud)
    plane_normal = mrob.registration.estimate_normal(plane_pointcloud)
    
    for i, p in enumerate(pnp_chessboard_points):
        print(p, plane_centroid, plane_normal)
        graph.add_factor_1pose_point2plane(z_point_x = p,
                                           z_point_y = plane_centroid,
                                           z_normal_y = plane_normal,
                                           nodePoseId = n1,
                                           obsInf = W)
        
    break

# solve the problem

graph.solve(mrob.LM)
result = graph.get_estimated_state()
print(result[0])

[-0.01185215 -0.10442502  1.28062433] [ 0.31284352 -0.14696845  1.22056683] [ 0.02847162 -0.16375913  0.9860894 ]
[ 0.04266775 -0.10413292  1.27337898] [ 0.31284352 -0.14696845  1.22056683] [ 0.02847162 -0.16375913  0.9860894 ]
[ 0.09718765 -0.10384082  1.26613364] [ 0.31284352 -0.14696845  1.22056683] [ 0.02847162 -0.16375913  0.9860894 ]
[ 0.15170756 -0.10354872  1.25888828] [ 0.31284352 -0.14696845  1.22056683] [ 0.02847162 -0.16375913  0.9860894 ]
[ 0.20622745 -0.10325662  1.25164294] [ 0.31284352 -0.14696845  1.22056683] [ 0.02847162 -0.16375913  0.9860894 ]
[ 0.26074736 -0.10296452  1.24439759] [ 0.31284352 -0.14696845  1.22056683] [ 0.02847162 -0.16375913  0.9860894 ]
[ 0.31526727 -0.10267241  1.23715224] [ 0.31284352 -0.14696845  1.22056683] [ 0.02847162 -0.16375913  0.9860894 ]
[ 0.36978715 -0.10238031  1.22990689] [ 0.31284352 -0.14696845  1.22056683] [ 0.02847162 -0.16375913  0.9860894 ]
[-0.01171332 -0.04952179  1.28388246] [ 0.31284352 -0.14696845  1.22056683] [ 0.02847162

In [7]:
# x = o3d.geometry.PointCloud()
# x.points = o3d.utility.Vector3dVector(t_scene_points)
# #o3d.visualization.draw_geometries([x])
# print(len(pnp_results))

In [6]:
# pi = mrob.registration.estimate_plane(t_scene_points[0])
# print(pi)

In [7]:
# test visually
pcx = o3d.geometry.PointCloud()
pcx.points= o3d.utility.Vector3dVector(X)
pcy = o3d.geometry.PointCloud()
pcy.points= o3d.utility.Vector3dVector(Y)
pcx.transform(result[0])
o3d.visualization.draw_geometries([pcx,pcy])